# Assoziationsanalyse

Der in diesem Notebook betrachtete Datensatz entspricht dem Datensatz aus den Folien, im Übungsmaterial finden Sie außerdem die Anwendung der Assoziationsanalyse auf einen größeren Datensatz. 

<hr style="border:1px solid gray"> </hr>

## Inhalt    

1. [Der Datensatz und die Pakete](#kap1)  
    1.1 [Apriori-Algorithmus mit mlxtend](#kap11)  
    1.2 [Apriori-Algorithmus mit efficient-apriori](#kap12)  
    
    
2. [Fazit](#kap2)   

<hr style="border:1px solid gray"> </hr>

## 1. Der Datensatz und die Pakete <a name="kap1"></a>

Der Datensatz wird nun zunächst eingelesen. Anders als in den vorhergehenden Notebooks liegen die Daten hier zunächst als Liste statt als Pandas-Dataframe vor. Diese wird vor Anwendung des Verfahrens in das passende Format konvertiert. 

In [ ]:
dataset = [['Apfel', 'Avocado', 'Chips'],
           ['Apfel', 'Avocado', 'Chips', 'Bier', 'Wasser'],
           ['Avocado', 'Chips', 'Bier'],
           ['Bier', 'Wasser'],
           ['Avocado', 'Chips', 'Wasser'],
           ['Apfel', 'Avocado', 'Wasser'],
           ['Apfel', 'Avocado', 'Chips'],
           ['Apfel', 'Avocado', 'Chips']
          ]

In [ ]:
type(dataset)

Bei einem größeren Geschäft oder Online-Shop mit vielen Artikeln entstehen durch die Dokumentation aller Einkäufe riesige Tabellen. Die Assoziationsanalyse mit dem Apriori-Algorithmus findet in diesen Listen effizient Assoziationsregeln, indem zunächt alle Kombinationen an Artikeln, deren Support größer als ein vorgegebener Wert ist, d.h. alle Artikelkombinationen, die "häufig" auftreten, herausgefiltert werden. Diese Kombinationen werden im Anschluss auf die enthaltenen Assoziationsregeln untersucht. 

Die Assoziationsanalyse ist nicht in der Python-Bibliothek scikit-learn enthalten, es gibt aber mehrere Pakete, um das Verfahren dennoch in Python nutzen zu können. Hier werden zwei unterschiedliche Möglichkeiten betrachtet: 

Die erste Möglichkeit ist das Paket mlxtend (http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/). Es kann mit dem Befehlt `!pip install mlxtend` installiert werden. 

Die zweite Möglichkeit ist das Paket efficient-appriori (https://efficient-apriori.readthedocs.io/en/latest/). Es kann mit dem Befehlt `!pip install efficient-apriori` installiert werden.

In beiden Fällen kann das Verfahren im Anschluss genutzt werden, indem die richtigen Befehle aus den Pakten importiert werden.

In [ ]:
!pip install mlxtend

In [ ]:
!pip install efficient-apriori

### 1.1 Apriori-Algorithmus mit mlxtend <a name="kap11"></a>

Die Daten sind wie bereits angesprochen als Listen der Einkäufe gegeben. Für die Assoziationsanalyse mit mlxtend ist eine etwas andere Darstellung praktischer: Es wird eine Tabelle gebildet, deren Spalten für die Produkte stehen und deren Zeilen für die einzelnen Einkäufe stehen. Ein Eintrag ist `True` (oder 1), falls der entsprechende Artikel in diesem Einkauf enthalten ist, und sonst `False` (oder 0). Um die Tabelle in die richtige Form zu bringen, wird der `TransactionEncoder()` genutzt. Anschließend wird die Variable wie bekannt in einen Pandas-Dataframe überführt:

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
te = TransactionEncoder()
te_matrix = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_matrix, columns=te.columns_)
print(df)

Durch dieses Vorgehen entstehen gerade bei einer großen Auswahl an Produkten eine deutlich größere Variable, in der aber viele Einträge `False` bzw. 0 sind, da jeder Einkauf nur einen Bruchteil des eigentlichen Angebots enthält. Die entstehende Variable muss also sehr effizient verarbeitet werden. 

Nachfolgend wird das Verfahren zunächst aus der Bibliothek mlxtend importiert und dann alle Kombinationen mit einem Support > 0.5 ausgegeben. 

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

frequent_itemsets

In `frequent_itemsets` sind nun die Artikel und Artikelkombinationen und deren Support-Werte gespeichert, die einen Support größer als `min_support` (0.5) haben. Nur diese müssen für das Aufstellen der gesuchten Assoziationsregeln überhaupt weiter betrachtet werden.

Geeignete Werte für `min_support` hängen stark vom Datensatz ab; zum Beispiel ist bei einem großen Online-Shop mit zehntausenden Artikeln nicht davon auszugehen, dass irgendein Support einen Wert von 0.5 erreicht, sondern `min_support` müsste viel kleiner gewählt werden.

Als zweite Eigenschaft für eine gesuchte Assoziationsregel wird gefordert, dass die Konfidenz für eine Regel "A $\rightarrow$ B" groß genug ist. Die Konfidenz lässt sich aus den bereits bekannten Support-Werten berechnen und das geschieht mit der Funktion `association_rules`. Dabei wird wiederum ein Grenzwert benötigt, was "groß genug" bedeutet, dies wird über den Parameter `min_threshold` angegeben.

Nachfolgend wird das Verfahren zunächst aus der Bibliothek mlxtend importiert und dann alle Kombinationen mit einer Konfidenz > 0.5 ausgegeben. 

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
rules

Die Assoziationsregeln stehen in den Spalten "antecedents" (= Vorläufer) und "consequents" (= Folgerungen). Die erste Regel ist z.B. "Apfel $\rightarrow$ Avocado", d.h. "wer einen Apfel kauft, kauft wahrscheinlich auch eine Avocado". 

Neben dem Support und der Konfidenz gibt es noch weitere Kenngrößen, anhand derer unterschieden werden kann, ob ein Zusammenhang stark genug ist, um es als Assoziationsregel in Betracht zu ziehen. Dazu gehören Lift, Leverage und Conviction (siehe weitere Spalten), mehr hierzu finden Sie im Zusatzmaterial. 

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 

In dem Pandas-DataFrame `rules` kann durch die korrekte Angabe von Bedingungen auf die Variable `rules['antecedents']` nun die Assoziationsregeln zu einem bestimmten Produkt des Datensatzes ausgegeben werden. Geben Sie alle Assoziationsregeln von der Produktmenge {Apfel} aus. 

Tipp: Betrachten Sie `rules['antecedents'] == ({'Apfel'})` .
</div>

In [ ]:
# Platz für Arbeitsauftrag

### 1.2 Apriori-Algorithmus mit efficient-apriori <a name="kap12"></a>

Bei der Nutzung des Verfahrens mit dem efficient-apriori können die Daten unmittelbar als Liste der Einkäufe übergeben werden und es genügt ein Befehl zur Bestimmung der Assoziationsregeln, dem sowohl der Mindestsupport als auch die Mindestkonfidenz mitgegeben werden. Das Verfahren efficient-apriori ist etwas schneller, die Ausgabe der Assoziationsregeln, nachfolgend gespeichert in der Variable `rules_ea`, ist allerdings etwas unübersichtlicher. 

Nachfolgend wird das Verfahren zunächst aus der Bibliothek efficient-apriori importiert und dann alle Kombinationen mit einem Support > 0.5 und einer Konfidenz > 0.5 ausgegeben. 

In [ ]:
from efficient_apriori import apriori
itemsets, rules_ea = apriori(dataset, min_support=0.5,  min_confidence=0.8)

In [ ]:
rules_ea

Die oben direkt in der Tablle mitausgegebenen Werte Support und Konfidenz, sowie beispielsweise Lift und Conviction, können wür jeden Eintrag der Variable `rules` angezeigt werden. Sie entsprechen bei korrekter Lösung des Arbeitsauftrages den Werten der ersten Zeile in der oben angegebenen Tabelle. 

In [ ]:
print(rules_ea[0].support)
print(rules_ea[0].confidence)
print(rules_ea[0].lift)

## 2. Fazit <a name="kap2"></a>

Es wurden auf zwei Wegen der Apriori-Algorithmus auf den Beispieldatensatz aus dem Video angewendet, hierbei wurde...

- festgestellt, dass Verfahren, die nicht in den üblichen Bibliotheken sind, meist über ein zusätzliches Paket gefunden werden können,


- festgestellt, dass verschiedene Ausführungen desselben Verfahrens sich in Laufzeit, Eingabe und Ausgabe unterscheiden können,


- wiederholt, dass die wichtigstens Hyperparameter der Mindestsupport und die Mindestkonfidenz sind, 


- beobachtet, dass es weitere Parameter im Kontext des Verfahrens gibt (bspw. Lift), welche im Übungsmaterial genauer betrachtet werden. 